In [ ]:
import requests, httpx
from httpx import Client
import os
import json
from pydantic import BaseModel, ValidationError
import polars as pl

# base url = https://www.roberthalf.com/us/en/insights/salary-guide/technology
BASE_DIR = os.getcwd()
OUTPUT_PATH = "Output"
LOCATION = ['Austin, TX','Houston, TX','San Diego, CA','San Jose, CA', 'Atlanta, GA','Dallas, TX','Fort Worth, TX','San Francisco, CA','Seattle, WA','New York, NY']
ACCOUNTING_CATEGORY = ['Corporate Accounting', 'Compliance', 'Tax', 'Internal Audit', 'IT Audit', 'Financial Reporting', 'General Accounting', 'Cost Accounting', 'Financial Analysis', 'Budget Analysis', 'Treasury Analysis', 'Business Analysis', 'Data Analysis', 'Business Intelligence Analysis', 'Business Systems Analysis', 'Financial Systems', 'Operational Support: Credit and Collections', 'Operational Support: Bookkeepers', 'Accounting Operations', 'Accounting Operations: Payroll', 'Public Accounting', 'Banking and Financial Markets', 'Financial Services', 'Financial Services: Financial Planning and Analysis (FP&A)', 'Financial Services: Internal Audit', 'Financial Services: Regulatory Reporting', 'Financial Services: Risk Analysis', 'Financial Services: Compliance', 'Fund Accounting and Operations']
TECHNOLOGY_CATEGORY = ['Executive Leadership','Software and Applications Development','Web Development','Consulting and Enterprise Systems','AI, Machine Learning and Data Science','Data: Engineering, Analytics and Reporting','Quality Assurance (QA), Training and Audit','Networking, Cloud and Systems','Security','IT Operations and Support']


def get_pricing (job_cat:str, specialization: str, location:str, master_df:pl.DataFrame, session:Client) -> pl.DataFrame:


    url:str = 'https://www.roberthalf.com/bin/salaryCategorySearchServlet'

    params = {
        'country': os.getenv('COUNTRY', 'US'),
        'language': os.getenv('LANGUAGE', 'en'),
        'specialization': os.getenv('SPECIALIZATION', specialization),
        'jobCategory': os.getenv('JOB_CATEGORY', job_cat),
        'preview': os.getenv('PREVIEW', 'false'),
        'isSEO': os.getenv('IS_SEO', 'false'),
        'normalization': os.getenv('NORMALIZATION', 'false'),
        'requestFrom': os.getenv('REQUEST_FROM', 'roleDetailsPage'),
        'location': os.getenv('LOCATION', location)
    }

    # Use a session to manage cookies securely

    response = session.get(url, params=params)

    print(response.status_code)
    try:
        response_json = response.json()

        # Define a Pydantic model to validate and parse the response JSON
        class SalaryData(BaseModel):
            jobTitle: str
            salaryLow: int
            salaryHigh: int
            salaryMidpoint: int

        # Parse the JSON into Pydantic objects
        salary_data_list = []
        for item in response_json.get('relatedPositions', []):
            try:
                salary_data = SalaryData(**item)
                salary_data_list.append(salary_data.model_dump())
            except ValidationError as e:
                print(f"Validation error: {e}")

        # Convert the Pydantic objects into a Polars DataFrame
        if salary_data_list:
            each_cat_result = pl.DataFrame(salary_data_list)
            each_cat_result = each_cat_result.with_columns(pl.lit(job_cat).alias('job_cat'), pl.lit(location).alias('location'),pl.lit(specialization).alias('specialization'))

            print (each_cat_result)

            master_df = master_df.vstack(each_cat_result)
        else:
            print("No valid salary data available.")

    except ValueError:
        print("Response is not in JSON format")
    
    return each_cat_result, master_df


def main():
    session = Client()
    master_df = pl.DataFrame()

    for each_location in LOCATION[:]:
        for each_cat in ACCOUNTING_CATEGORY:
            location = each_location
            specilization = 'Finance and Accounting'
            # specilization = 'Technology'

            each_cat_result, master_df = get_pricing(job_cat = each_cat, specialization=specilization,location= location,master_df= master_df,session=session)
            each_cat = each_cat.replace(':',' ')
        print ('Complete:',each_location)
        file_name = f'result {location} {specilization}.xlsx'
        master_df.write_excel(workbook = os.path.join(BASE_DIR,OUTPUT_PATH,file_name))


if __name__ == '__main__':
    main()    